In [18]:
from company_loader import get_company_data
from company import Company

# Buyer side
data = get_company_data("UNP")  # Union Pacific

if data:
    acquirer = Company(
        name=data["name"],
        revenue=data["revenue"],
        ebitda=data["ebitda"],
        ebit_margin=data["ebitda_margin"],
        wacc=data["wacc"],
        shares_outstanding=data["shares_outstanding"]
    )

# Target side
data = get_company_data("NSC")  # Norfolk Southern

if data:
    target = Company(
        name=data["name"],
        revenue=data["revenue"],
        ebitda=data["ebitda"],
        ebit_margin=data["ebitda_margin"],
        wacc=data["wacc"],
        shares_outstanding=data["shares_outstanding"]
    )

print(acquirer)
print(target)

Union Pacific Corporation - Revenue: 24393.0M, EBITDA: 12366.0M, EBITDA Margin: 50.7%, WACC: 8.0%, Shares: 593.04M
Norfolk Southern Corporation - Revenue: 12178.0M, EBITDA: 6056.0M, EBITDA Margin: 49.7%, WACC: 8.0%, Shares: 224.35M


In [19]:
class SynergyModel:
    def __init__(
        self, 
        acquirer, 
        target, 
        cost_synergy_pct,           
        revenue_synergy_pct,        
        synergy_ebit_margin,        # Estimated EBIT margin for synergistic revenues (e.g., 20%)
        integration_cost,           # Amount in millions
        duration_years
    ):
        self.acquirer = acquirer
        self.target = target
        self.cost_synergy_pct = cost_synergy_pct
        self.revenue_synergy_pct = revenue_synergy_pct
        self.synergy_ebit_margin = synergy_ebit_margin
        self.integration_cost = integration_cost
        self.duration_years = duration_years # Number of years over which synergies are realized
        self.discount_rate = acquirer.wacc  # WACC of the acquirer for discounting

    def discounted_value(self, amount):
        return sum(amount / ((1 + self.discount_rate) ** t) for t in range(1, self.duration_years + 1))

    def compute(self):
        # Combined data
        combined_revenue = self.acquirer.revenue + self.target.revenue
        combined_costs = (
            self.acquirer.revenue * (1 - self.acquirer.ebit_margin) +
            self.target.revenue * (1 - self.target.ebit_margin)
        )

        # Synergies calculated based on percentages
        annual_cost_synergies = self.cost_synergy_pct * combined_costs
        annual_revenue_synergies = self.revenue_synergy_pct * combined_revenue * self.synergy_ebit_margin

        # Calculation using the discount rate
        pv_cost_synergies = self.discounted_value(annual_cost_synergies)
        pv_revenue_synergies = self.discounted_value(annual_revenue_synergies)
        pv_integration_cost = self.discounted_value(self.integration_cost)

        total_synergies = pv_cost_synergies + pv_revenue_synergies
        net_value_created = total_synergies - pv_integration_cost

        return {
            "Annual Cost Synergies": round(annual_cost_synergies, 2),
            "Annual Revenue Synergies (EBIT)": round(annual_revenue_synergies, 2),
            "PV Cost Synergies": round(pv_cost_synergies, 2),
            "PV Revenue Synergies": round(pv_revenue_synergies, 2),
            "PV Integration Cost": round(pv_integration_cost, 2),
            "Net Value Created": round(net_value_created, 2)
        }

    def compute_interval(self, delta_cost=0.01, delta_rev=0.01, delta_margin=0.02):
            """
            Calcule une fourchette [min, max] de la valeur nette créée.
            """
            scenarios = []
    
            # Creating variation combinations for different scenarios
            for dc in [-delta_cost, 0, delta_cost]:
                for dr in [-delta_rev, 0, delta_rev]:
                    for dm in [-delta_margin, 0, delta_margin]:
                        model = SynergyModel(
                            acquirer=self.acquirer,
                            target=self.target,
                            cost_synergy_pct=self.cost_synergy_pct + dc,
                            revenue_synergy_pct=self.revenue_synergy_pct + dr,
                            synergy_ebit_margin=self.synergy_ebit_margin + dm,
                            integration_cost=self.integration_cost,
                            duration_years=self.duration_years
                        )
                        value = model.compute()["Net Value Created"]
                        scenarios.append(value)
    
            return min(scenarios), max(scenarios)

In [20]:
from synergy_loader import load_synergy_benchmarks

benchmarks = load_synergy_benchmarks("mna_deals.xlsx")

model = SynergyModel(
    acquirer=acquirer,
    target=target,
    cost_synergy_pct=benchmarks["cost_synergy_pct"],
    revenue_synergy_pct=benchmarks["revenue_synergy_pct"],
    synergy_ebit_margin=benchmarks["synergy_ebit_margin"],
    integration_cost=benchmarks["integration_cost"],
    duration_years=benchmarks["duration_years"]
)

result = model.compute()
low, high = model.compute_interval()

for k, v in result.items():
    print(f"{k}: {v}M")

print("------------------")
print(f"Range of Value Creation (sensitivity): {low:.2f}M – {high:.2f}M")

Annual Cost Synergies: 889.35M
Annual Revenue Synergies (EBIT): 244.93M
PV Cost Synergies: 2945.65M
PV Revenue Synergies: 811.25M
PV Integration Cost: 2649.7M
Net Value Created: 1107.2M
------------------
Range of Value Creation (sensitivity): 176.58M – 2086.28M
